In [1]:
!git clone https://github.com/openai/gpt-2.git

Cloning into 'gpt-2'...
remote: Enumerating objects: 203, done.
remote: Total 203 (delta 0), reused 0 (delta 0), pack-reused 203
Receiving objects: 100% (203/203), 4.37 MiB | 14.82 MiB/s, done.
Resolving deltas: 100% (105/105), done.


In [0]:
import os
os.chdir("gpt-2")

In [3]:
!pip3 install -r requirements.txt

     |████████████████████████████████| 81kB 3.9MB/s 
     |████████████████████████████████| 604kB 10.4MB/s 
     |████████████████████████████████| 51kB 23.3MB/s 
  Created wheel for fire: filename=fire-0.2.1-py2.py3-none-any.whl size=103527 sha256=fc21cc31820e0c34def8fa5a2c3885dd953b8b77831c79c10d073e5a4bae10a9
  Stored in directory: /root/.cache/pip/wheels/31/9c/c0/07b6dc7faf1844bb4688f46b569efe6cafaa2179c95db821da
  Created wheel for regex: filename=regex-2017.4.5-cp36-cp36m-linux_x86_64.whl size=533178 sha256=d5ac1535cf67687f6c77e36b1a23774fd805ee81854ee686af77a21035824191
  Stored in directory: /root/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built fire regex
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [4]:
!python3 download_model.py 345M

Fetching checkpoint: 1.00kit [00:00, 843kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 51.5Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 922kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:21, 64.5Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 7.95Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 48.7Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 48.1Mit/s]                                                       


In [0]:
!export PYTHONIOENCODING=UTF-8

In [0]:
os.chdir('src')

In [0]:
import json
import os
import numpy as np
import tensorflow as tf
import model, sample, encoder

In [0]:
def interact_model(
    model_name,
    seed,
    nsamples,
    batch_size,
    length,
    temperature,
    top_k,
    models_dir
):
    models_dir = os.path.expanduser(os.path.expandvars(models_dir))
    if batch_size is None:
        batch_size = 1
    assert nsamples % batch_size == 0

    enc = encoder.get_encoder(model_name, models_dir)
    hparams = model.default_hparams()
    with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if length is None:
        length = hparams.n_ctx // 2
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.Session(graph=tf.Graph()) as sess:
        context = tf.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        output = sample.sample_sequence(
            hparams=hparams, length=length,
            context=context,
            batch_size=batch_size,
            temperature=temperature, top_k=top_k
        )

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
        saver.restore(sess, ckpt)

        while True:
            raw_text = input("Model prompt >>> ")
            while not raw_text:
                print('Prompt should not be empty!')
                raw_text = input("Model prompt >>> ")
            context_tokens = enc.encode(raw_text)
            generated = 0
            for _ in range(nsamples // batch_size):
                out = sess.run(output, feed_dict={
                    context: [context_tokens for _ in range(batch_size)]
                })[:, len(context_tokens):]
                for i in range(batch_size):
                    generated += 1
                    text = enc.decode(out[i])
                    print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
                    print(text)
            print("=" * 80)

In [9]:
interact_model(
    '345M',
    None,
    1,
    1,
    300,
    1,
    0,
    '/content/gpt-2/models'
)

W0822 14:01:15.747022 139686458316672 deprecation_wrapper.py:119] From /content/gpt-2/src/sample.py:33: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

W0822 14:01:15.748268 139686458316672 deprecation_wrapper.py:119] From /content/gpt-2/src/model.py:148: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0822 14:01:15.760870 139686458316672 deprecation_wrapper.py:119] From /content/gpt-2/src/model.py:152: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W0822 14:01:15.806949 139686458316672 deprecation_wrapper.py:119] From /content/gpt-2/src/model.py:36: The name tf.rsqrt is deprecated. Please use tf.math.rsqrt instead.

W0822 14:01:21.063520 139686458316672 deprecation.py:323] From /content/gpt-2/src/sample.py:46: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0822 14

Model prompt >>> I went to a lounge to celebrate my birthday and 
======================================== SAMPLE 1 ========================================
 OP became brought in from Uni, as I figured the guys that were there would know who OP was, in case I forgot or something. Being Protestant in devil land, Smart two West res friends got a bit bored going between the guys, I just wanted to chase it and did. I ended up following another OP (who forgot OP's name) and was like those guys op said OP was looking for, he was ecstatic and wrote.
 What I love about tumblr is there are many genres we have a bit of fun extremelyif you are into things like my Navy background was cool lol but seriously people benefit is being able to share their interests and this club does in many cases. Anyone here that is into plants, ice cream or family is in, at least 35% of the population of lsu and it really help for people who now live full time in town because their ability to pursue what they are pas

KeyboardInterrupt: ignored